# Importing libraries and loading data

In [1]:
# Install Python packages using pip.

# The "!pip" command allows you to run shell commands in Jupyter Notebook or Colab cells.
# It is used here to install Python packages.
# The "-q" flag stands for "quiet," which means it will suppress output during installation.
# "feature_engine," "autoviz," and "dataprep" are the packages being installed.
# The "2>/dev/null" part redirects any error messages (stderr) to the null device, effectively silencing them.
# This is often used when you want to hide installation messages.
!pip install -q feature_engine autoviz dataprep 2>/dev/null

In [3]:
# Import necessary libraries
import numpy as np  # Import NumPy for handling numerical operations
import pandas as pd  # Import Pandas for data manipulation and analysis
import warnings  # Import Warnings to suppress unnecessary warnings

# Suppress warning messages
warnings.filterwarnings("ignore")

# Import AutoViz from the autoviz library for automated visualization of data
from autoviz import AutoViz_Class

# Import load_dataset and create_report from the dataprep library for data loading and EDA
from dataprep.datasets import load_dataset
from dataprep.eda import create_report

# Import SHAP for interpreting model predictions
import shap

# Import matplotlib for data visualization
import matplotlib.pyplot as plt

# Import CatBoostRegressor for building a regression model
from catboost import Pool, CatBoostRegressor

# Import mean_squared_error for evaluating model performance
from sklearn.metrics import mean_squared_error

# Import train_test_split for splitting the data into training and testing sets
from sklearn.model_selection import train_test_split

# Import RareLabelEncoder from feature_engine.encoding for encoding categorical features
from feature_engine.encoding import RareLabelEncoder

# Import CountVectorizer from sklearn.feature_extraction.text for text feature extraction
from sklearn.feature_extraction.text import CountVectorizer

# Import ast and re for working with text and regular expressions
import ast
import re

# Import gc for garbage collection
import gc

# Set Pandas options to display a maximum of 1000 rows
pd.set_option('display.max_rows', 1000)

In [4]:
%%time

# Load the raw data
# taken from https://www.kaggle.com/code/lorentzyeung/starter-notebook-for-uk-property-price-paid-data

colnames=['Transaction_unique_identifier', 'price', 'Date_of_Transfer', 
          'postcode', 'Property_Type', 'Old/New', 
          'Duration', 'PAON', 'SAON', 
          'Street', 'Locality', 'Town/City', 
          'District', 'County', 'PPDCategory_Type',
          'Record_Status - monthly_file_only'
          ] 

df = pd.read_csv('/kaggle/input/price-paid-data-202304/202304.csv',
                  header=None,
                 names=colnames,
                 infer_datetime_format=True,
                 parse_dates=["Date_of_Transfer"],
                 dayfirst=False
                 ).drop(['Transaction_unique_identifier'], axis=1)

# select properties only from Greater London county
# df = df[df['County']=="GREATER LONDON"]

# Reads the CSV file into a Pandas DataFrame
item0 = df.shape[0]  # Stores the initial number of rows in the DataFrame
df = df.drop_duplicates()  # Removes duplicate rows from the DataFrame
item1 = df.shape[0]  # Stores the number of rows after removing duplicates
print(f"There are {item0-item1} duplicates found in the dataset")  # Prints the number of duplicates that were removed



There are 14827 duplicates found in the dataset
CPU times: user 2min 53s, sys: 21.4 s, total: 3min 14s
Wall time: 3min 57s


In [10]:
# df = df.set_index(keys, drop=True, append=False, inplace=False, verify_integrity=False)

# df['log10_price'] = df['price']
# .apply(lambda x: np.log10(x))
# df['Year'] = df['Date_of_Transfer'].dt.year
# select data from 2013-2023
df_remove_date = df[df['Date_of_Transfer']>'2013-01-01']

In [33]:
# 15 columns
# only use 12 columns to download into 7 csv files
df_remove_date.head(5)
# df_remove_date.info()

,price,Date_of_Transfer,postcode,Property_Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPDCategory_Type,Record_Status - monthly_file_only
18221746,620000,2013-02-14,W9 1BE,F,N,L,44G,NaN,RANDOLPH AVENUE,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A
18221747,250000,2013-02-11,L39 8ST,D,N,F,PARK HOUSE FARM,NaN,PLEX MOSS LANE,HALSALL,ORMSKIRK,WEST LANCASHIRE,LANCASHIRE,A,A
18221748,225000,2013-12-17,SW15 3HJ,F,N,L,"KERSFIELD HOUSE, 11",FLAT 2,KERSFIELD ROAD,PUTNEY,LONDON,WANDSWORTH,GREATER LONDON,A,A
18221749,141000,2013-09-27,EX5 2NG,T,N,F,2,NaN,BOUCHERS COTTAGES,CLYST HONITON,EXETER,EAST DEVON,DEVON,A,A
18221750,158000,2013-10-25,DN19 7BP,D,N,F,BELLA VISTA,NaN,GREEN LANE,NaN,BARROW-UPON-HUMBER,NORTH LINCOLNSHIRE,NORTH LINCOLNSHIRE,A,A


In [40]:

# df['Location'] = df['Street'].fillna('None') + ', ' + df['Locality'].fillna('None') + ', ' + df['Town/City'].fillna('None') + ', ' + df['District'].fillna('None')
# df['Location'] = df['Location'].str.lower()

selected_cols1 = ['Date_of_Transfer','price','Property_Type', 'Old/New', 'Duration']
selected_cols2 = ['Date_of_Transfer','postcode']
selected_cols3 = ['Date_of_Transfer','Street']

selected_cols4 = ['Date_of_Transfer','District']
selected_cols5 = ['Date_of_Transfer','Locality']
selected_cols6 = ['Date_of_Transfer','Town/City']
selected_cols7 = ['Date_of_Transfer','County', 'PPDCategory_Type']

# df1 = df_remove_date[selected_cols1]
df2 = df_remove_date[selected_cols2]
df3 = df_remove_date[selected_cols3]
df4 = df_remove_date[selected_cols4]
df5 = df_remove_date[selected_cols5]
df6 = df_remove_date[selected_cols6]
df7 = df_remove_date[selected_cols7]


In [45]:
df_remove_date


,price,Date_of_Transfer,postcode,Property_Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPDCategory_Type,Record_Status - monthly_file_only
18221746,620000,2013-02-14,W9 1BE,F,N,L,44G,NaN,RANDOLPH AVENUE,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A
18221747,250000,2013-02-11,L39 8ST,D,N,F,PARK HOUSE FARM,NaN,PLEX MOSS LANE,HALSALL,ORMSKIRK,WEST LANCASHIRE,LANCASHIRE,A,A
18221748,225000,2013-12-17,SW15 3HJ,F,N,L,"KERSFIELD HOUSE, 11",FLAT 2,KERSFIELD ROAD,PUTNEY,LONDON,WANDSWORTH,GREATER LONDON,A,A
18221749,141000,2013-09-27,EX5 2NG,T,N,F,2,NaN,BOUCHERS COTTAGES,CLYST HONITON,EXETER,EAST DEVON,DEVON,A,A
18221750,158000,2013-10-25,DN19 7BP,D,N,F,BELLA VISTA,NaN,GREEN LANE,NaN,BARROW-UPON-HUMBER,NORTH LINCOLNSHIRE,NORTH LINCOLNSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28276223,490000,2023-01-13,RM1 4SR,T,N,F,12,NaN,HEATHER GLEN,NaN,ROMFORD,HAVERING,GREATER LONDON,A,A
28276224,365000,2023-01-10,SS2 4UA,F,N,L,"CENTENARY PLACE, 1",FLAT 22,SOUTHCHURCH BOULEVARD,NaN,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,A,A
28276225,730000,2023-01-05,CO13 0EY,S,N,F,HIGH VIEW,NaN,LITTLE CLACTON ROAD,GREAT HOLLAND,FRINTON-ON-SEA,TENDRING,ESSEX,A,A
28276226,379000,2023-01-06,CO15 5NA,D,N,F,12,NaN,DULWICH ROAD,HOLLAND-ON-SEA,CLACTON-ON-SEA,TENDRING,ESSEX,A,A


In [51]:
k = df_remove_date.reindex(columns=['Date_of_Transfer','price','Property_Type', 'Old/New', 
                                    'Duration','Street','District','Locality','Town/City', 'PPDCategory_Type'])
k.tail(5)

,Date_of_Transfer,price,Property_Type,Old/New,Duration,Street,District,Locality,Town/City,PPDCategory_Type
28276223,2023-01-13,490000,T,N,F,HEATHER GLEN,HAVERING,NaN,ROMFORD,A
28276224,2023-01-10,365000,F,N,L,SOUTHCHURCH BOULEVARD,SOUTHEND-ON-SEA,NaN,SOUTHEND-ON-SEA,A
28276225,2023-01-05,730000,S,N,F,LITTLE CLACTON ROAD,TENDRING,GREAT HOLLAND,FRINTON-ON-SEA,A
28276226,2023-01-06,379000,D,N,F,DULWICH ROAD,TENDRING,HOLLAND-ON-SEA,CLACTON-ON-SEA,A
28276227,2023-01-05,450000,D,N,F,INGRAMS PIECE,TENDRING,ARDLEIGH,COLCHESTER,A


In [52]:
# Filter the data for the desired time range (2000 to 2023)

# Save the filtered data to a new CSV file
# df1.to_csv('UK_housing_prices_1a.csv', index=False)
# df2.to_csv('UK_housing_prices_2d.csv', index=False)
# df3.to_csv('UK_housing_prices_3d.csv', index=False)
df4.to_csv('UK_housing_prices_4d.csv', index=False)
df5.to_csv('UK_housing_prices_5d.csv', index=False)
df6.to_csv('UK_housing_prices_6d.csv', index=False)
df7.to_csv('UK_housing_prices_7d.csv', index=False)


In [41]:
df = df[selected_cols]
# print(df.shape)
# df.sample(5).T

In [13]:
df_filtered

,log10_price,Year,Property_Type,Old/New,Duration,Location,PPDCategory_Type


In [9]:
df.columns

Index(['log10_price', 'Year', 'Property_Type', 'Old/New', 'Duration',
       'Location', 'PPDCategory_Type'],
      dtype='object')

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
df['Location'].value_counts().head(30)

# Data visualisation

In [ ]:
# An update taken from the nice work https://www.kaggle.com/code/anshtanwar/auto-eda-missing-migrants-interactive-charts 
# made by @anshtanwar

# Import the AutoViz_Class
# This class is used for automated exploratory data analysis and visualization.
AV = AutoViz_Class()

# Initialize variables
filename = ""  # Specify the filename of the dataset (empty in this case)
target_variable = 'log10_price'  # Specify the target variable for analysis
custom_plot_dir = "custom_plot_directory"  # Specify the directory to save custom plots

# Perform automated EDA using the AutoViz library
# The following parameters are used:
# - filename: Empty in this case as the data is provided directly as 'df'
# - sep: Delimiter used in the data (comma in this case)
# - depVar: Target variable for analysis ('rating' in this case)
# - dfte: DataFrame to be analyzed ('df' is assumed to be defined earlier)
# - header: Indicates that the first row contains column names (0 for True)
# - verbose: Verbosity level (1 for verbose output)
# - lowess: Smoothing using Lowess algorithm (False for no smoothing)
# - chart_format: Format in which charts will be generated (HTML format in this case)
# - max_rows_analyzed: Maximum number of rows to analyze (up to 10,000 rows)
# - max_cols_analyzed: Maximum number of columns to analyze (up to 50 columns)
# - save_plot_dir: Directory to save the generated plots ('custom_plot_directory' in this case)
dft = AV.AutoViz(
    filename,
    sep=",",
    depVar=target_variable,
    dfte=df,
    header=0,
    verbose=1,
    lowess=False,
    chart_format="html",
    max_rows_analyzed=min([df.shape[0], 10**4]),
    max_cols_analyzed=min([df.shape[1], 50]),
    save_plot_dir=custom_plot_dir
)

In [ ]:
# Import the necessary library for displaying HTML content
from IPython.core.display import display, HTML

# Import the pathlib library to work with file paths
from pathlib import Path

# Initialize an empty list to store file names
file_names = []

# Use pathlib to iterate through HTML files in a specific directory
for file in Path(f'/kaggle/working/{custom_plot_dir}/{target_variable}/').glob('*.html'):
    
    # Extract the filename from the full path and add it to the list
    filename = str(file).split('/')[-1]
    file_names.append(filename)

# Iterate through the list of file names and display each HTML file
for file_name in file_names:
    
    # Construct the full file path for each HTML file
    file_path = f'/kaggle/working/{custom_plot_dir}/{target_variable}/{file_name}'
    
    # Open the HTML file for reading
    with open(file_path, 'r') as file:
        
        # Read the content of the HTML file
        html_content = file.read()
        
        # Display the HTML content using IPython
        display(HTML(html_content))

In [ ]:
create_report(df.sample(10**4))

# Data transformation

In [ ]:
# Accessing DataFrame columns
# This line of code retrieves the column names from a DataFrame called 'df'.
# It allows you to access and work with the names of the columns in the DataFrame.

df.columns

In [ ]:
df.sample(5).T

In [ ]:
# Display information about the DataFrame 'df'
# This includes the data types, non-null values, and memory usage
# Useful for getting a quick overview of the dataset's structure
df.info()

In [ ]:
%%time

# Select the main label.
main_label = 'log10_price'

# Set up a rare label encoder for selected columns.
for col in ['Property_Type', 'Old/New', 'Duration', 'Location', 'PPDCategory_Type']:
    df[col] = df[col].fillna('None')
    encoder = RareLabelEncoder(n_categories=1, max_n_categories=150, replace_with='Other', tol=50.0 / df.shape[0])
    df[col] = encoder.fit_transform(df[[col]])
    print(f"LabelEncoded column {col}")

print(df.shape)  # Print the shape of the resulting DataFrame.
df.sample(5).T  # Display a sample of 5 rows, transposed for easier readability.

In [ ]:
gc.collect()

# Machine learning